# Preprocesamiento de datos

## Ejercicios: Parte I

<div class="alert-warning">
    
Indicaciones: Los ejercicios 1, 2, 3 y 4 deben leer su data de links en Github y el código y los resultados deben ser publicados en páginas en Github.

</div>

## Ejercicio 1

<div class="alert-success">
    
1. Use two maps of points.

2. Compute the distance matrix for both maps.

3. Select one row of the distance matrix, and plot the two points with the minimal distance on top of the country of your choosing. (distancia entre punto a punto)
</div>

Mapas de puntos: Plantas de energía solar y eólica en China
Fuente: [Global Power Plant Database](https://datasets.wri.org/dataset/globalpowerplantdatabase)

In [139]:
import pandas as pd
import geopandas as gpd
import os

PowerPlants = pd.read_csv("https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/data/global_power_plant_database.csv", dtype={'other_fuel3': 'object'})

China_PowerPlants = PowerPlants[PowerPlants["country"]=="CHN"]
China_PowerPlants = China_PowerPlants.loc[:,["name","gppd_idnr","capacity_mw","latitude","longitude","primary_fuel"]]
China_PowerPlants.reset_index(drop=True, inplace=True)

#Dataframe a GeoDataframe:
China_PowerPlants_gpd = gpd.GeoDataFrame(data=China_PowerPlants.copy(),
                                        geometry=gpd.points_from_xy(x=China_PowerPlants.longitude,
                                        y=China_PowerPlants.latitude),
                                        crs=4326)

#Cambiando los CRSs por unos proyectados y que coincidan con los de China3415:
China3415_PowerPlants = China_PowerPlants_gpd.to_crs(3415)

#Mapas de puntos: Plantas de energía solar y eólica
China3415_SolarPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Solar"]
China3415_SolarPowerPlants = China3415_SolarPowerPlants.rename(columns={"name":"SolarPowerPlant_Name"})
China3415_SolarPowerPlants = China3415_SolarPowerPlants.reset_index()
China3415_SolarPowerPlants.drop(columns="index",inplace=True)


China3415_WindPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Wind"]
China3415_WindPowerPlants = China3415_WindPowerPlants.rename(columns={"name":"WindPowerPlant_Name"})
China3415_WindPowerPlants = China3415_WindPowerPlants.reset_index()
China3415_WindPowerPlants.drop(columns="index",inplace=True)


# Geopackage con los 2 mapas de puntos:
China3415_SolarPowerPlants.to_file(os.path.join("maps","China3415","China3415_Maps.gpkg"), layer="SolarPower_Plants", driver="GPKG",index="")
China3415_WindPowerPlants.to_file(os.path.join("maps","China3415","China3415_Maps.gpkg"), layer="WindPower_Plants", driver="GPKG",index="")

## Ejercicio 2

<div class="alert-success">
    
1. Use a map of points and a map of lines.

2. Compute the distance matrix for both maps.

3. Select one line of the distance matrix, and plot the closests and the farthest point to that line. (Se puede seleccionar un río o un sistema y calcular el punto más cercano o lejano a la línea. Si se escoge un punto, se calcula la línea más cercana y lejana)
    
    
</div>

Plantas de energía nuclear en China:

In [140]:
China3415_NuclearPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Nuclear"]
China3415_NuclearPowerPlants = China3415_NuclearPowerPlants.rename(columns={"name":"NuclearPowerPlant_Name"})
China3415_NuclearPowerPlants = China3415_NuclearPowerPlants.reset_index()
China3415_NuclearPowerPlants.drop(columns="index",inplace=True)

China3415_NuclearPowerPlants.to_file(os.path.join("maps","China3415","China3415_Maps.gpkg"), layer="NuclearPower_Plants", driver="GPKG",index="")

Ríos principales en China:
Fuente: [Major Rivers in the World](https://datacatalog.worldbank.org/search/dataset/0042032/Major-Rivers-of-the-World)

In [141]:
#Mapa de ríos principales (más extensos) en el mundo
MajorRiverslink = "https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/maps/World_MajorRivers/MajorRivers.shp"
MajorRivers = gpd.read_file(MajorRiverslink)
MajorRivers3415 = MajorRivers.to_crs(3415)

#Mapa de China
countries = gpd.read_file("https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/maps/World_Countries/World_Countries.shp")
China = countries[countries.COUNTRY=='China']
China3415 = China.to_crs(3415)
China3415.to_file(os.path.join("maps","China3415","China3415_Maps.gpkg"), layer="Country", driver="GPKG",index="")

#Recortar los ríos para que solo se encuentren los que encierran a China
China3415_MajorRivers = gpd.clip(gdf = MajorRivers3415, mask = China3415)
China3415_MajorRivers = China3415_MajorRivers.reset_index()
China3415_MajorRivers = China3415_MajorRivers.drop(columns="index")
China3415_MajorRivers = China3415_MajorRivers.rename(columns={"NAME":"River_Name"})
China3415_MajorRivers = China3415_MajorRivers.rename(columns={"NAME":"River_Name", "SYSTEM":"System"})
China3415_MajorRivers.replace({None:"None"}, inplace = True)

China3415_MajorRivers.to_file(os.path.join("maps","China3415","China3415_Maps.gpkg"), layer="Major_Rivers", driver="GPKG",index="")

## Ejercicio 3

<div class="alert-success">
    
1. Create a set of points and a set of polygons

2. Compute the distance matrix for both sets.

3. Select one polygon of the distance matrix, and plot the closests and the farthest point to that polygon. (cualquiera de los dos mapas. Puede ser el mapa estático o el mapa interactivo)
    
</div>   

Conjunto de polígonos: Convex hulls de ríos acorde al sistema al que pertenecen. Los que no forman parte de un sistema de ríos, se han engoblado en un mismo convex hull.

In [142]:
China3415_MajorRivers

,River_Name,System,MILES,KILOMETERS,geometry
0,Mekong,None,2942.45095,4735.42525,"MULTILINESTRING ((-1411216.206 1988600.080, -1..."
1,Irrawaddy,None,1297.67231,2088.40532,"LINESTRING (-1057977.328 1318106.869, -1052909..."
2,Xi Jiang,None,1258.96091,2026.10524,"MULTILINESTRING ((-511985.712 1072122.053, -52..."
3,Salween,None,1795.43718,2889.48183,"LINESTRING (-1659115.629 1889755.161, -1659001..."
4,Yangtze,None,3513.65086,5654.68424,"LINESTRING (-1707905.143 2144013.773, -1645915..."
5,Huang He,None,3371.55586,5426.00404,"MULTILINESTRING ((-1189346.745 2174180.647, -1..."
6,Amur,None,3199.18145,5148.59377,"MULTILINESTRING ((1197526.833 3878331.408, 119..."
7,Brahmaputra,None,1897.09807,3053.08950,"LINESTRING (-2589752.461 1864433.403, -2569624..."
8,Indus,None,2015.16218,3243.09564,"LINESTRING (-2775061.963 2154664.832, -2775051..."
9,Irtysh,Ob,3099.37017,4987.96276,"LINESTRING (-1887621.291 3824322.599, -1868389..."


In [143]:
#GeoDataFrame, agrupando por ríos
China3415_RiverSystems = China3415_MajorRivers.dissolve(by="System")
China3415_RiverSystems = China3415_RiverSystems.drop(columns = ["River_Name","MILES","KILOMETERS"])
China3415_RiverSystems.to_file(os.path.join("maps","China3415","China3415_Maps.gpkg"), layer='RiverSystems', driver="GPKG",index="")

#Convertir a convex hull y luego a un GeoDataFrame con polígonos
China3415_RiverSystems_hulls = China3415_RiverSystems.convex_hull.to_frame()
China3415_RiverSystems_hulls.rename(columns = {0:"geometry"}, inplace = True)
China3415_RiverSystems_hulls = China3415_RiverSystems_hulls.set_geometry("geometry")
China3415_RiverSystems_hulls

China3415_RiverSystems_hulls.to_file(os.path.join("maps","China3415","China3415_Maps.gpkg"), layer='RiverSystems_hulls', driver="GPKG",index="")

Conjunto de puntos: Plantas de energía nuclear en China

## Ejercicio 4

<div class="alert-success">
    
1. Create a set of points and a set of lines

2. Get the buffer for the lines, select different values for the distance. Jugar con varias distancias, pero solo colocar una que tenga resultados interesantes en el notebook final.

3. Keep the points that are within the buffer (as in point 2, you need to play with differn distances until you show something interesting. Todos los puntos que están dentro del buffer creado para la línea.
   

</div>   

- Set de puntos: China3415_NuclearPowerPlants
- Set de líneas: China3415_RiverSystems